# Agent

Ask Bert
\begin{align}
p(r_t,u_t,\omega_t,a_t | \omega_{t-1}) \propto \underbrace{p(r_t|u_t) p(u_t|\omega_t)}_{\text{likelihood}} \underbrace{p(\omega_t | \omega_{t-1}, a_t)}_{\text{state transition}}\underbrace{p(a_t)}_{\text{control}} \qquad (1)
\end{align}

\begin{align}
p(r_t,u_t,\omega_t | \omega_{t-1}) \propto p(r_t|\omega_t)p(\omega_t | \omega_{t-1}, u_t)p(u_t) \qquad (2)
\end{align}

\begin{align}
p(r_t,u_t,\omega_t \mid u_{t-1}) \propto \underbrace{p(r_t \mid u_t)}_{\text{response}} \underbrace{ p(u_t \mid u_{t-1}) p(u_t \mid \omega_t)}_{\text{perception }p(u_t \mid u_{t-1},\omega_t)} \underbrace{p(\omega_t)}_{\substack{\text{control} \\ \text{prior}}}  \qquad (3)
\end{align}

In [4]:
using ForneyLab
using ProgressMeter
using LinearAlgebra
using Plots

In [5]:
pad(sym::Symbol, t::Int) = sym*:_*Symbol(lpad(t,3,'0'))

pad (generic function with 1 method)

In [87]:
# Model (1)
n_samples = 1
fg = FactorGraph()

# State prior
@RV ω_0 ~ GaussianMeanPrecision(placeholder(:m_ω_0), placeholder(:w_ω_0))


# Transition and observation model
ω = Vector{Variable}(undef, n_samples)
a = Vector{Variable}(undef, n_samples)
u = Vector{Variable}(undef, n_samples)
p = Vector{Variable}(undef, n_samples)
r = Vector{Variable}(undef, n_samples)

ω_i_min = ω_0
for i in 1:n_samples

    @RV a[i] ~ Gamma(3.0, 2.0)
    @RV ω[i] ~ GaussianMeanPrecision(ω_i_min, a[i])
    @RV u[i] ~ GaussianMeanPrecision(ω[i], 100.0)
    
    f(z) = 1/(1+ exp(-z))
    @RV p[i] ~ Nonlinear{Sampling}(u[i], g=f)
    @RV r[i] ~ Bernoulli(p[i])

    # Data placeholder
    placeholder(r[i], :r, index=i)

    # Reset state for next step
    ω_i_min = ω[i]
end
q = PosteriorFactorization([ω_0; ω], p, a, ids=[:Ω :P :A])
algo = messagePassingAlgorithm(free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true)
println(source_code)

begin

function stepP!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 4))

messages[1] = ruleVBGaussianMeanPrecisionOut(nothing, marginals[:ω_1], ProbabilityDistribution(Univariate, PointMass, m=100.0))
messages[2] = ruleVBBernoulliIn1(ProbabilityDistribution(Univariate, PointMass, m=data[:r][1]), nothing)
messages[3] = ruleSPNonlinearSIn1MN(f, messages[2], nothing)
messages[4] = ruleSPNonlinearSOutNM(f, nothing, messages[1])

marginals[:p_1] = messages[4].dist * messages[2].dist
marginals[:u_1] = messages[1].dist * messages[3].dist

return marginals

end

function stepA!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBGammaOut(nothing, ProbabilityDistribution(Univariate, PointMass, m=3.0), ProbabilityDistribution(Univariate, PointMass, m=2.0))
messages[2] = ruleSVBGaussianMeanPrecisionW(marginals[:ω_1_ω_0], nothing)

marginals[:a_1] = messages[1].dist * messages[2].dist

return marginals


In [19]:
# Model (2)
n_samples = 1
fg = FactorGraph()

# State prior
@RV ω_0 ~ GaussianMeanPrecision(placeholder(:m_ω_0), placeholder(:w_ω_0))

# Transition and observation model
ω = Vector{Variable}(undef, n_samples)
u = Vector{Variable}(undef, n_samples)
p = Vector{Variable}(undef, n_samples)
r = Vector{Variable}(undef, n_samples)

ω_i_min = ω_0
for i in 1:n_samples

    @RV [id=pad(:u, i)] u[i] ~ GaussianMeanPrecision(placeholder(:m_u_*i), placeholder(:w_u_*i))
    
    f(x, y) = 1/(1+ exp(-(x+y)))

    @RV ω[i] ~ Nonlinear{Sampling}(u[i], ω_i_min, g=f)
    
    @RV r[i] ~ GaussianMeanVariance(ω[i], 0.1)

    # Data placeholder
    placeholder(r[i], :r, index=i)

    # Reset state for next step
    ω_i_min = ω[i]
end
draw()
q = PosteriorFactorization()
algo = messagePassingAlgorithm(free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true)
println(source_code)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 6338127045655304815 -->
 
 6338127045655304815 
 
 placeholder_w_u_1 
 
<!-- 3213410709260195288 -->
 
 3213410709260195288 
 
 clamp_1 
 
<!-- 15984978427604282737 -->
 
 15984978427604282737 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 15984978427604282737--6338127045655304815 -->
 
 15984978427604282737--6338127045655304815 
 
 w_u_1 
 1 out 
 3 w 
 
<!-- 1914987922651644954 -->
 
 1914987922651644954 
 
 placeholder_m_u_1 
 
<!-- 15984978427604282737--1914987922651644954 -->
 
 15984978427604282737--1914987922651644954 
 
 m_u_1 
 1 out 
 2 m 
 
<!-- 6876054326587716001 -->
 
 6876054326587716001 
 
 placeholder_r_1 
 
<!-- 157303045624423255 -->
 
 157303045624423255 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 6876054326587716001--157303045624423255 -->
 
 6876054326587716001--157303045624423255 
 
 r_1 
 1 out 
 1 out 
 
<!-- 17156998999720565854 -->
 
 17156998999720565854 
 
 placeholder_w_ω_0 
 
<!-- 3223127505606627611 -->
 
 3223127505606627611 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 3223127505606627611--17156998999720565854 -->
 
 3223127505606627611--17156998999720565854 
 
 w_ω_0 
 1 out 
 3 w 
 
<!-- 1867666355217926804 -->
 
 1867666355217926804 
 
 placeholder_m_ω_0 
 
<!-- 3223127505606627611--1867666355217926804 -->
 
 3223127505606627611--1867666355217926804 
 
 m_ω_0 
 1 out 
 2 m 
 
<!-- 16090444182833958737 -->
 
 16090444182833958737 
 
 g{Sampling} 
 nonlinear_1 
 
<!-- 16090444182833958737--15984978427604282737 -->
 
 16090444182833958737--15984978427604282737 
 
 u_001 
 1 out 
 2 in1 
 
<!-- 16090444182833958737--3223127505606627611 -->
 
 16090444182833958737--3223127505606627611 
 
 ω_0 
 1 out 
 3 in2 
 
<!-- 157303045624423255--3213410709260195288 -->
 
 157303045624423255--3213410709260195288 
 
 clamp_1 
 1 out 
 3 v 
 
<!-- 157303045624423255--16090444182833958737 -->
 
 157303045624423255--16090444182833958737 
 
 ω_1 
 1 out 
 2 m

begin

function init()

messages = Array{Message}(undef, 6)

messages[1] = Message(vague(GaussianMeanPrecision))
messages[2] = Message(vague(GaussianMeanPrecision))

return messages

end

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 6))

messages[1] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Univariate, PointMass, m=data[:m_u_1]), Message(Univariate, PointMass, m=data[:w_u_1]))
messages[2] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Univariate, PointMass, m=data[:m_ω_0]), Message(Univariate, PointMass, m=data[:w_ω_0]))
messages[3] = ruleSPNonlinearSOutNGX(f, nothing, messages[1], messages[2])
messages[4] = ruleSPGaussianMeanVarianceMPNP(Message(Univariate, PointMass, m=data[:r][1]), nothing, Message(Univariate, PointMass, m=0.1))
messages[5] = ruleSPNonlinearSInGX(f, 2, messages[4], messages[1], messages[2])
messages[6] = ruleSPNonlinearSInGX(f, 1, messages[4], messages[1], messages[2])

marginals[:u_001] = messag

In [20]:
eval(Meta.parse(source_code))

freeEnergy (generic function with 1 method)

In [21]:
using JLD
dataJLD = JLD.load("data_model_2.jld")

Dict{String,Any} with 3 entries:
  "x" => [0.230003, 0.499106, 0.827884, 0.871674, 0.788024, 0.688976, 0.495929,…
  "r" => [0, 1, 1, 1, 1, 0, 0, 0, 0, 1  …  1, 1, 1, 0, 1, 0, 1, 1, 1, 0]
  "u" => [0.0221682, -0.233577, 1.0716, 1.08795, 0.441383, 0.00731298, -0.70525…

In [31]:
f(x, y) = 1/(1+ exp(-(x+y)))

f (generic function with 1 method)

In [30]:
messages = init()
marginals = Dict()
for i in 1:length(dataJLD["x"])
    data = Dict(:m_u_1 => 0.0, :w_u_1 => 1.0, :m_ω_0 => 0.0, :w_ω_0 => 1.0, :r => dataJLD["r"][i])
    step!(data, marginals,  messages)
end

LoadError: MethodError: no method matching exp(::Array{ForwardDiff.Dual{ForwardDiff.Tag{ForneyLab.var"#log_joint#297"{Array{Float64,1},Diagonal{Float64,Array{Float64,1}},ProbabilityDistribution{Univariate,GaussianMeanVariance},typeof(f),Array{Int64,1}},Float64},Float64,2},1})
Closest candidates are:
  exp(!Matched::Variable) at /Users/albertpod/.julia/dev/ForneyLab/src/factor_nodes/exponential.jl:37
  exp(!Matched::BigFloat) at mpfr.jl:603
  exp(!Matched::Float16) at math.jl:1144
  ...

In [152]:
# Model (3) fails due to multiple inputs of different distributions
n_samples = 1
fg = FactorGraph()

# State prior

@RV u_0 ~ GaussianMeanPrecision(placeholder(:m_u_0), placeholder(:w_u_0))

# Transition and observation model
ω = Vector{Variable}(undef, n_samples)
p = Vector{Variable}(undef, n_samples)
u = Vector{Variable}(undef, n_samples)
r = Vector{Variable}(undef, n_samples)

u_i_min = u_0
for i in 1:n_samples

    @RV ω[i] ~ Beta(1.0, 1.0)
    
    @RV u[i] ~ GaussianMeanPrecision(u_i_min, 100.0)
    
    sigmoid(x, z) = 1/(1+ exp(-(x+z)))
    @RV p[i] ~ Nonlinear{Sampling}(u[i], ω[i], g=sigmoid)
    @RV r[i] ~ Bernoulli(p[i])

    # Data placeholder
    placeholder(r[i], :r, index=i)

    # Reset state for next step
    u_i_min = u[i]
end
draw()
q = PosteriorFactorization(ω, u, ids=[:Ω :U])
algo = messagePassingAlgorithm(free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 14148136209743170974 -->
 
 14148136209743170974 
 
 clamp_2 
 
<!-- 9635957328786935328 -->
 
 9635957328786935328 
 
 Beta 
 beta_1 
 
<!-- 9635957328786935328--14148136209743170974 -->
 
 9635957328786935328--14148136209743170974 
 
 clamp_2 
 1 out 
 3 b 
 
<!-- 6914634977457005216 -->
 
 6914634977457005216 
 
 clamp_1 
 
<!-- 9635957328786935328--6914634977457005216 -->
 
 9635957328786935328--6914634977457005216 
 
 clamp_1 
 1 out 
 2 a 
 
<!-- 9247697078806554825 -->
 
 9247697078806554825 
 
 clamp_3 
 
<!-- 1168489843262766985 -->
 
 1168489843262766985 
 
 placeholder_w_u_0 
 
<!-- 4165278879402515816 -->
 
 4165278879402515816 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 4165278879402515816--1168489843262766985 -->
 
 4165278879402515816--1168489843262766985 
 
 w_u_0 
 1 out 
 3 w 
 
<!-- 14995289286316231626 -->
 
 14995289286316231626 
 
 placeholder_m_u_0 
 
<!-- 4165278879402515816--14995289286316231626 -->
 
 4165278879402515816--14995289286316231626 
 
 m_u_0 
 1 out 
 2 m 
 
<!-- 16096405115069515812 -->
 
 16096405115069515812 
 
 g{Sampling} 
 nonlinear_1 
 
<!-- 16096405115069515812--9635957328786935328 -->
 
 16096405115069515812--9635957328786935328 
 
 ω_1 
 1 out 
 3 in2 
 
<!-- 8421455896784479204 -->
 
 8421455896784479204 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 16096405115069515812--8421455896784479204 -->
 
 16096405115069515812--8421455896784479204 
 
 u_1 
 1 out 
 2 in1 
 
<!-- 6140178622539777057 -->
 
 6140178622539777057 
 
 placeholder_r_1 
 
<!-- 10738412901592838949 -->
 
 10738412901592838949 
 
 Ber 
 bernoulli_1 
 
<!-- 6140178622539777057--10738412901592838949 -->
 
 6140178622539777057--10738412901592838949 
 
 r_1 
 1 out 
 1 out 
 
<!-- 10738412901592838949--16096405115069515812 -->
 
 10738412901592838949--16096405115069515812 
 
 p_1 
 1 out 
 2 p 
 
<!-- 8421455896784479204--9247697078806554825 -->
 
 8421455896784479204--9247697078806554825 
 
 clamp_3 
 1 out 
 3 w 
 
<!-- 8421455896784479204--4165278879402515816 -->
 
 8421455896784479204--4165278879402515816 
 
 u_0 
 1 out 
 2 m

LoadError: No applicable SumProductRule{Nonlinear{Sampling}} update for Nonlinear{Sampling} node with inbound types: Message{Beta,var_type} where var_type<:ForneyLab.VariateType, Message{GaussianMeanPrecision,var_type} where var_type<:ForneyLab.VariateType, Nothing

In [154]:
# Model (3) hack around previous error
n_samples = 1
fg = FactorGraph()

# State prior
@RV u_0 ~ GaussianMeanPrecision(placeholder(:m_u_0), placeholder(:w_u_0))

# Transition and observation model
ω = Vector{Variable}(undef, n_samples)
ωsample = Vector{Variable}(undef, n_samples)
ωn = Vector{Variable}(undef, n_samples)
p = Vector{Variable}(undef, n_samples)
u = Vector{Variable}(undef, n_samples)
un = Vector{Variable}(undef, n_samples)
utr = Vector{Variable}(undef, n_samples)
r = Vector{Variable}(undef, n_samples)

u_i_min = u_0
for i in 1:n_samples

    @RV ω[i] ~ Beta(1.0, 1.0)
    βN(x) = x
    @RV ωsample[i] ~ Nonlinear{Sampling}(ω[i], g=βN)
    @RV ωn[i] ~ GaussianMeanPrecision(ωsample[i], 100.0)

    @RV utr[i] ~ GaussianMeanPrecision(u_i_min, 10.0)
    sumβN(x, y) = x + y
    @RV u[i] ~ Nonlinear{Sampling}(ωn[i], utr[i], g=sumβN)
    @RV un[i] ~ GaussianMeanPrecision(u[i], 100.0)
    sigmoid(x) = 1/(1+ exp(-(x)))
    @RV p[i] ~ Nonlinear{Sampling}(un[i], g=sigmoid)
    @RV r[i] ~ GaussianMeanPrecision(p[i], 1.0)

    # Data placeholder
    placeholder(r[i], :r, index=i)

    # Reset state for next step
    u_i_min = un[i]
end
q = PosteriorFactorization(ω, ωn, [u_0; un], ids=[:Ω :ΩN :U])
algo = messagePassingAlgorithm(free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true)
println(source_code)

begin

function stepU!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 6))

messages[1] = ruleVBGaussianMeanPrecisionOut(nothing, marginals[:u_1], ProbabilityDistribution(Univariate, PointMass, m=100.0))
messages[2] = ruleVBGaussianMeanPrecisionM(ProbabilityDistribution(Univariate, PointMass, m=data[:r][1]), nothing, ProbabilityDistribution(Univariate, PointMass, m=1.0))
messages[3] = ruleSPNonlinearSIn1MN(sigmoid, messages[2], nothing)
messages[4] = ruleVBGaussianMeanPrecisionOut(nothing, ProbabilityDistribution(Univariate, PointMass, m=data[:m_u_0]), ProbabilityDistribution(Univariate, PointMass, m=data[:w_u_0]))
messages[5] = ruleVBGaussianMeanPrecisionM(marginals[:utr_1], nothing, ProbabilityDistribution(Univariate, PointMass, m=10.0))
messages[6] = ruleSPNonlinearSOutNM(sigmoid, nothing, messages[1])

marginals[:p_1] = messages[6].dist * messages[2].dist
marginals[:u_0] = messages[4].dist * messages[5].dist
marginals[:un_1] = messages[1].dist * 